---
title: Bearing's mathematics
subtitle: Computing the direction in respect to the north pole
date: 2024-08-25
categories: [tutorial, geospatial, mathematics]
image: images/cover.png
toc: true
draft: true
colab: <a href="https://colab.research.google.com/github/SebastianoF/GeoDsBlog/blob/master/posts/gds-2024-08-25-bearing/index.ipynb" target="_blank"><img src="images/colab.svg"></a>
github: <a href="https://github.com/SebastianoF/GeoDsBlog/blob/master/posts/gds-2024-08-25-bearing/index.ipynb" target="_blank">  <img src="images/github.svg"> </a>
twitter-card:
  image: images/cover.png
---


![](images/cover.png)


## Introduction

No, this post is not about how to stand the [part of physics where experiments are cheap](https://www.math.fsu.edu/~wxm/Arnold.htm#:~:text=Mathematics%20is%20the%20part%20of,be%20discovered%20with%20less%20expense.).

Today's topic is a computation that solves a recurring problem in geospatial data science, arising frequently when dealing with [AIS (Automatic Identification System)](https://en.wikipedia.org/wiki/Automatic_identification_system) data.

Often considered along with the [Haversine distance](https://sebastianof.github.io/GeoDsBlog/posts/gds-2024-01-10-haversine-dist/), the *bearing* at the point $A$ in the direction of another point $B$ is the angle $\beta$ between the meridian passing through $A$ and the geodesic connecting $A$ to $B$. 

For maritime applications, the *bearing of a vessel* is the angle between the meridian and the direction where the vessel is intending to go, measured from zero to $360$ degrees clockwise. It differs from the *heading*, that is the angle between the meridian and the actual vessel orientation; heading and bearing may or may not be coincident due to currents, winds and route adjustments.

This blog post is organized into three parts. The first and the second are about the bearing formula for the sphere and the ellipsoid model respectively. These two first part have no code, and are only about the mathematical formulae. The third part is the only one needed if you are here for the python code.

- @sec-spherical-model Bearings for the Earth's spherical model
- @sec-ellipsoid-model Bearings for the Earth's ellipsoidal model
- @sec-coding Python code


## Bearings for the Earth's spherical model {#sec-spherical-model}

Latitude and longitude, traditionally expressed in degrees, are renamed in radians with the conventional greek letters $\theta$ (theta) and $\varphi$ (phi):

$$
\begin{align*}
\theta &:= \text{rad}(\text{Lon}) \\
\varphi &:= \text{rad}(\text{Lat}) \\
\end{align*}
$$ 

To project a point $(\theta, \varphi)$ with $\theta \in [-\pi, \pi]$ and $\varphi \in [-\pi/2, \pi/2]$, on the sphere of radius $R$ we use the immersion:

$$
\iota: \mathbb{S}^2 \rightarrow  \mathbb{R}^3
$$ 

$$
\iota(\theta, \varphi) = \begin{cases}
       x = R \cos\varphi \cos\theta\\
       y = R \cos\varphi \sin\theta\\
       z = R \sin\varphi\\
     \end{cases}
$$ 

that is the usual spherical coordinate system [also presented in the post](https://sebastianof.github.io/GeoDsBlog/posts/gds-2024-01-10-haversine-dist/#fig-spherical-coordinates) about the Haversine distance.


<!-- Image here -->


<!-- Image here -->

Now, given two points on the sphere $A = (\theta_A, \varphi_A)$ and $B = (\theta_B, \varphi_B)$, the bearing $\beta$ from $A$ to $B$ is the angle between the plane passing through $A$, the north pole $N$ and the origin $O$ (indicated with $\pi(OAN)$), and the plane $A$, $B$ and $O$ (indicated with $\pi(OAB)$). 

The angle between two plane coincides with the angle between the two normals, that can be computed with the exterior products[^1]:

$$
~\text{ Vector } \perp \text{ to } \pi(OAN) \text{ is } \hat{n}_N^A = \frac{N\times A}{ \left\Vert N\times A \right\Vert } \\
~ \\
~\text{ Vector } \perp \text{ to } \pi(OAB) \text{ is } \hat{n}_B^A = \frac{B\times A}{ \left\Vert B\times A \right\Vert }  \\
$$ 

To compute the bearing is therefore the simple computation of the angle between $\hat{n}_N^A $ and $\hat{n}_B^A $. This can be done via the scalar product:

$$
\hat{n}_B^A \cdot \hat{n}_N^A = \left\Vert \hat{n}_B^A \right\Vert  \left\Vert \hat{n}_N^A \right\Vert \cos \beta
$$

which gives us $\beta = \arccos (\hat{n}_B^A \cdot \hat{n}_N^A) $.

If we were to persevere into this direction, we would face the issue that the cosine is invertible only between $0$ and $\pi$, so we would lose all the bearings pointing westwards. We can solve this problem adding a sign that changes according to the direction of the normals, and have 2 different formulae, depending on the cardinal direction.


[^1]: you can also change the of the point and take $A\times N$ and $A\times B$, if you prefer, as long as $A$ is consistently the first element or the second element in both products.

... Fig 2 here ...

Instead of moving towards this direction, we have a better option, that consists of starting with the observation that $\beta$ is invariant under the rotations around the $z$ axis. So we can rotate the sphere to make $\tilde{\theta}_A = 0$ and $\tilde{\theta}_B = \theta_B - \theta_A $. In this reference $N \times A$ is parallel to the $y$ axis, and $\beta$ is the angle between $\hat{n}_B^A$ and the $y$-axis.

Now cosine of $\beta$ is the projection of $\hat{n}_B^A$ on the $y$ axis, which is its $y$-component, and the sine of $\beta$ is the projection of $\hat{n}_B^A$ on the plane $xz$, that is the square root of the sum of squared $x$ and $z$ components of $\hat{n}_B^A$. 

In formula, for $B\times A = ((B\times A)_x, (B\times A)_y, (B\times A)_z )$:

$$
\tan\beta = \frac{\sin\beta}{\cos\beta} = \frac{\sqrt{ (B\times A)_x^2 + (B\times A)_z^2 }}{ (B\times A)_y^2}
$$

Now it is only a matter of computing $B\times A$ in the rotated coordinate system, and to invert the tangent.


The 3 points $N, A, B$ before re-orienting the axis have coordinates:

$$
\begin{align*}
N &= (0,0,1) \\
A &= (R\cos\varphi_A\cos\theta_A, R\cos\varphi_A\sin\theta_A, R\sin\varphi_A) \\
B &= (R\cos\varphi_B\cos\theta_B, R\cos\varphi_B\sin\theta_B, R\sin\varphi_B)
\end{align*}
$$

After the re-orientation $\theta_A = 0$ and $\tilde{\theta}_B = \theta_B - \theta_A$, we have:

$$
\begin{align*}
N &= (0,0,1) \\
A &= (R\cos\varphi_A, 0, R\sin\varphi_A) \\
B &= (R\cos\varphi_B\cos\tilde{\theta}_B, R\cos\varphi_B\sin\tilde{\theta}_B, R\sin\varphi_B)
\end{align*}
$$

Now to compute $\tan\beta$ we compute first

TODO from HERE!

<!-- 
![](images/earth_models.png)

https://math.libretexts.org/Bookshelves/Abstract_and_Geometric_Algebra/Introduction_to_Groups_and_Geometries_(Lyons)/03%3A_Geometries/3.04%3A_Elliptic_geometry

https://geodaesie.info/images/zfv/147-jahrgang-2022/downloads/zfv_2022_3_Lapaine.pdf -->



<!-- last part: -->

$$
\beta(A, B) = \text{atan2} ( y, x )
$$ 

where

$$
\begin{align*}
y &:= \cos\varphi_B \sin(\theta_B - \theta_A) \\
x &:= \cos\varphi_A \sin\varphi_B - \sin\varphi_A \cos\varphi_B \cos(\theta_B - \theta_A) \\
\end{align*}
$$ 

and where $\text{atan2}(y, x)$ is the arctangent of $y/x$, evaluated for any $x$ and $y$ as

$$
\text{atan2}(y, x) 
= 
\begin{cases}
  \arctg(y/x) \phantom{+ \pi} \qquad x>0, ~~\forall y  \\
  \arctg(y/x) + \pi \qquad x<0, ~~ y \geq 0  \\
  \arctg(y/x) - \pi \qquad x<0, ~~ y < 0  \\
  +\frac{\pi}{2} \phantom{\arctg(y/x)} \qquad x = 0, ~~ y > 0  \\
  -\frac{\pi}{2} \phantom{\arctg(y/x)} \qquad x = 0, ~~ y < 0  \\
  \text{undefined} \qquad \qquad x = 0, ~~ y = 0  \\
\end{cases}
$$




conclusion:

Bearing on the ellipsoid (Vincentry's Bearing), which is ...



## Bearings for the Earth's ellipsoidal model {#sec-ellipsoid-model}

## Python code {#sec-coding}



In [ ]:
# in progress